Load transformed data from a parquet file generated by the ETL.

In [1]:
import ibmos2spark
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-f260c7dd-ea22-450d-ac4f-411d7bbe11a8',
    'IBM_API_KEY_ID': 'fN2y36SJQsBNLN7BhTsrFNjuEIaYuqSsJn5059Il-meW',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'advanceddatasciencecapstone-donotdelete-pr-eegj6wm2sruxqi',
    'FILE': 'arxiv_data_transformed.parquet'
}

configuration_name = 'os_e245254457804a5abc5f0819c8c11f69_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials_1, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json


df_arxiv_data = spark.read.parquet(cos.url('arxiv_data_transformed.parquet', 'advanceddatasciencecapstone-donotdelete-pr-eegj6wm2sruxqi'))

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200830144900-0002
KERNEL_ID = d76277a0-2720-4f07-a887-d52d83edb1e0


ValueError: Invalid input: credentials. endpoint is required!

Define ranking function to rank articles against a given set of keywords. Higher rank indicates higher similarity.

In [7]:
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql import types as T
from pyspark.sql.functions import udf

def rank(doc, keywords):
    doc_keys = DenseVector(doc.indices)
    doc_keys = list([int(x) for x in doc_keys])
    
    doc_values = DenseVector(doc.values)
    doc_values = list([float(x) for x in doc_values])
    
    rank = 0.0
    for keyword in keywords:
        keyword_rank = 0.0
        if keyword in doc_keys:
            keyword_index = doc_keys.index(keyword)
            keyword_rank = doc_values[keyword_index]
        rank += keyword_rank
        
    return rank

def sparse_to_array(v):
    v = DenseVector(v)
    new_array = list([float(x) for x in v])
    return new_array


udf_rank = udf(rank, T.FloatType())

Define function to get 5 be

In [5]:
import pyspark.sql.functions as SF
from pyspark.sql.functions import desc
from operator import itemgetter

def get_best_matches(article_name):
    my_entry = df_arxiv_data.where(df_arxiv_data.title == article_name)
    my_keywords = my_entry.select("title_vec").collect()[0][0].indices.tolist()
    my_keywords_importance = my_entry.select("title_vec").collect()[0][0].values.tolist()
    my_keywords_and_importances = list(zip(my_keywords, my_keywords_importance))

    my_keywords_and_importances.sort(key=itemgetter(1), reverse=True)
    my_most_important_keys = [d[0] for d in my_keywords_and_importances[:5]]

    df_arxiv_data_ranked = df_arxiv_data.withColumn('rank', udf_rank(df_arxiv_data['title_vec'], SF.array([SF.lit(x) for x in my_most_important_keys])))
    best_matches = df_arxiv_data_ranked.orderBy(desc("rank")).take(5)
    
    return best_matches

In [6]:
best_matches = get_best_matches('Dynamic Routing Between Capsules')

NameError: name 'df_arxiv_data' is not defined